<a href="https://colab.research.google.com/github/evitts1989/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Corey_Evitts_LS_DS_213.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv', keep_default_na=False)

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
df.shape

(23040, 21)

In [6]:
df.describe()

,BLOCK,LOT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,1.624479,1.328133e+06
std,3713.269887,628.025255,0.803898,1.025394e+07
min,1.000000,1.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,4.000000,8.500000e+08


In [7]:
df.dtypes

BOROUGH                           object
NEIGHBORHOOD                      object
BUILDING_CLASS_CATEGORY           object
TAX_CLASS_AT_PRESENT              object
BLOCK                              int64
LOT                                int64
EASE-MENT                         object
BUILDING_CLASS_AT_PRESENT         object
ADDRESS                           object
APARTMENT_NUMBER                  object
ZIP_CODE                          object
RESIDENTIAL_UNITS                 object
COMMERCIAL_UNITS                  object
TOTAL_UNITS                       object
LAND_SQUARE_FEET                  object
GROSS_SQUARE_FEET                 object
YEAR_BUILT                        object
TAX_CLASS_AT_TIME_OF_SALE          int64
BUILDING_CLASS_AT_TIME_OF_SALE    object
SALE_PRICE                         int64
SALE_DATE                         object
dtype: object

In [8]:
df['BOROUGH'].isnull().sum()

0

In [9]:
df['SALE_PRICE'].dtypes

dtype('int64')

In [10]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011,1,0,1,"10,733",1979,2007,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,,O5,144 WEST 37TH STREET,,10018,0,6,6,"2,962",15435,1920,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,,O5,40 WEST 38TH STREET,,10018,0,7,7,"2,074",11332,1930,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,,R4,"1 SHERIDAN SQUARE, 8C",8C,10014,1,0,1,0,500,0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,,R1,"20 EAST 65TH STREET, B",B,10065,1,0,1,0,6406,0,2,R1,0,01/01/2019


**Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.**


In [11]:
df2 = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000) & (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')]

df2.shape

(3151, 21)

In [12]:
df2

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,,A9,4832 BAY PARKWAY,,11230,1,0,1,"6,800",1325,1930,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,,A1,80-23 232ND STREET,,11427,1,0,1,"4,000",2001,1940,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,,A1,1260 RHINELANDER AVE,,10461,1,0,1,"3,500",2043,1925,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,,A1,469 E 25TH ST,,11226,1,0,1,"4,000",2680,1899,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,,A5,5521 WHITTY LANE,,11203,1,0,1,"1,710",1872,1940,1,A5,620000,01/02/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,OTHER,01 ONE FAMILY DWELLINGS,1,13215,3,,A2,244-15 135 AVENUE,,11422,1,0,1,"3,300",1478,1925,1,A2,635000,04/30/2019
23031,4,OTHER,01 ONE FAMILY DWELLINGS,1,11612,73,,A1,10919 132ND STREET,,11420,1,0,1,"2,400",1280,1930,1,A1,514000,04/30/2019
23032,4,OTHER,01 ONE FAMILY DWELLINGS,1,11808,50,,A0,135-24 122ND STREET,,11420,1,0,1,"4,000",1333,1945,1,A0,635000,04/30/2019
23033,4,OTHER,01 ONE FAMILY DWELLINGS,1,12295,23,,A1,134-34 157TH STREET,,11434,1,0,1,"2,500",1020,1935,1,A1,545000,04/30/2019


**Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.**


In [13]:
train= df2[(df2['SALE_DATE'] >= '01/01/2018')&(df2['SALE_DATE'] < '04/01/2019')]
train.shape

(2507, 21)

In [14]:
test= df2[(df2['SALE_DATE'] >= '04/01/2019')&(df2['SALE_DATE'] <= '04/30/2019')]
test.shape

(644, 21)

**Do one-hot encoding of categorical features.**

In [15]:
train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,2507.0,6758.303949,3975.909029,21.0,3837.5,6022.0,9888.5,16350.0
LOT,2507.0,75.778221,157.531138,1.0,21.0,42.0,70.0,2720.0
TAX_CLASS_AT_TIME_OF_SALE,2507.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
SALE_PRICE,2507.0,621573.742321,291607.225676,104000.0,440500.0,560000.0,750000.0,1955000.0


In [16]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507
EASE-MENT,2507,1,,2507
TAX_CLASS_AT_PRESENT,2507,2,1,2476
APARTMENT_NUMBER,2507,2,,2506
RESIDENTIAL_UNITS,2507,2,1,2476
COMMERCIAL_UNITS,2507,3,0,2467
TOTAL_UNITS,2507,4,1,2436
BOROUGH,2507,5,4,1204
NEIGHBORHOOD,2507,6,OTHER,2382
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919


In [17]:
test.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,644.0,7539.559006,3854.438406,107.0,4500.0,7393.0,10857.5,16350.0
LOT,644.0,76.773292,174.379223,1.0,21.0,41.0,68.0,2202.0
TAX_CLASS_AT_TIME_OF_SALE,644.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
SALE_PRICE,644.0,655757.054348,296979.954119,107500.0,470000.0,599300.0,781625.0,1912500.0


In [18]:
test.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
BUILDING_CLASS_CATEGORY,644,1,01 ONE FAMILY DWELLINGS,644
EASE-MENT,644,1,,644
APARTMENT_NUMBER,644,1,,644
TAX_CLASS_AT_PRESENT,644,2,1,635
COMMERCIAL_UNITS,644,2,0,635
TOTAL_UNITS,644,3,1,625
RESIDENTIAL_UNITS,644,3,1,634
BOROUGH,644,5,4,376
NEIGHBORHOOD,644,6,OTHER,608
BUILDING_CLASS_AT_TIME_OF_SALE,644,10,A1,267


In [19]:
train['BOROUGH'].value_counts()

4    1204
5     662
3     398
2     242
1       1
Name: BOROUGH, dtype: int64

In [20]:
train.groupby('BOROUGH')['SALE_PRICE'].mean()

BOROUGH
1    739000.000000
2    473104.623967
3    768734.633166
4    649005.172757
5    537305.802115
Name: SALE_PRICE, dtype: float64

In [0]:
target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'LAND_SQUARE_FEET', 'SALE_DATE', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE']
features = train.columns.drop([target] + high_cardinality)

In [0]:
X_train = train[features]
y_train = train['SALE_PRICE']
X_test = test[features]
y_test = test['SALE_PRICE']

In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)

In [0]:
X_test = encoder.transform(X_test)

In [25]:
X_train

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,EASE-MENT_,APARTMENT_NUMBER_,APARTMENT_NUMBER_RP.,ZIP_CODE_11230,ZIP_CODE_11427,ZIP_CODE_10461,ZIP_CODE_11226,ZIP_CODE_11203,ZIP_CODE_11229,ZIP_CODE_11364,ZIP_CODE_11373,ZIP_CODE_11365,ZIP_CODE_11429,ZIP_CODE_11414,ZIP_CODE_11369,ZIP_CODE_11415,ZIP_CODE_11413,ZIP_CODE_11434,ZIP_CODE_11435,ZIP_CODE_10312,ZIP_CODE_10308,ZIP_CODE_10314,ZIP_CODE_11236,ZIP_CODE_11228,...,YEAR_BUILT_2007,YEAR_BUILT_2009,YEAR_BUILT_1959,YEAR_BUILT_1921,YEAR_BUILT_1984,YEAR_BUILT_2011,YEAR_BUILT_1987,YEAR_BUILT_1979,YEAR_BUILT_1975,YEAR_BUILT_1924,YEAR_BUILT_1995,YEAR_BUILT_1978,YEAR_BUILT_2001,YEAR_BUILT_1927,YEAR_BUILT_1962,YEAR_BUILT_1890,YEAR_BUILT_1999,YEAR_BUILT_1963,YEAR_BUILT_1969,YEAR_BUILT_1991,YEAR_BUILT_2008,YEAR_BUILT_2015,YEAR_BUILT_1957,YEAR_BUILT_2012,YEAR_BUILT_1936,YEAR_BUILT_1933,YEAR_BUILT_1949,YEAR_BUILT_1929,YEAR_BUILT_2010,YEAR_BUILT_1926,YEAR_BUILT_1973,YEAR_BUILT_1939,YEAR_BUILT_1982,YEAR_BUILT_1932,YEAR_BUILT_2014,YEAR_BUILT_1900,YEAR_BUILT_1944,YEAR_BUILT_2013,YEAR_BUILT_1941,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5495,801,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
61,0,1,0,0,0,1,0,0,0,0,0,1,1,0,7918,72,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
78,0,0,1,0,0,1,0,0,0,0,0,1,1,0,4210,19,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
108,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5212,69,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
111,1,0,0,0,0,1,0,0,0,0,0,1,1,0,7930,121,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,0,0,0,1,0,1,0,0,0,0,0,1,1,0,4081,44,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18130,0,0,0,1,0,1,0,0,0,0,0,1,1,0,2373,201,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18132,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1132,42,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18134,0,0,0,1,0,1,0,0,0,0,0,1,1,0,3395,37,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [26]:
X_test

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,EASE-MENT_,APARTMENT_NUMBER_,APARTMENT_NUMBER_RP.,ZIP_CODE_11230,ZIP_CODE_11427,ZIP_CODE_10461,ZIP_CODE_11226,ZIP_CODE_11203,ZIP_CODE_11229,ZIP_CODE_11364,ZIP_CODE_11373,ZIP_CODE_11365,ZIP_CODE_11429,ZIP_CODE_11414,ZIP_CODE_11369,ZIP_CODE_11415,ZIP_CODE_11413,ZIP_CODE_11434,ZIP_CODE_11435,ZIP_CODE_10312,ZIP_CODE_10308,ZIP_CODE_10314,ZIP_CODE_11236,ZIP_CODE_11228,...,YEAR_BUILT_2007,YEAR_BUILT_2009,YEAR_BUILT_1959,YEAR_BUILT_1921,YEAR_BUILT_1984,YEAR_BUILT_2011,YEAR_BUILT_1987,YEAR_BUILT_1979,YEAR_BUILT_1975,YEAR_BUILT_1924,YEAR_BUILT_1995,YEAR_BUILT_1978,YEAR_BUILT_2001,YEAR_BUILT_1927,YEAR_BUILT_1962,YEAR_BUILT_1890,YEAR_BUILT_1999,YEAR_BUILT_1963,YEAR_BUILT_1969,YEAR_BUILT_1991,YEAR_BUILT_2008,YEAR_BUILT_2015,YEAR_BUILT_1957,YEAR_BUILT_2012,YEAR_BUILT_1936,YEAR_BUILT_1933,YEAR_BUILT_1949,YEAR_BUILT_1929,YEAR_BUILT_2010,YEAR_BUILT_1926,YEAR_BUILT_1973,YEAR_BUILT_1939,YEAR_BUILT_1982,YEAR_BUILT_1932,YEAR_BUILT_2014,YEAR_BUILT_1900,YEAR_BUILT_1944,YEAR_BUILT_2013,YEAR_BUILT_1941,TAX_CLASS_AT_TIME_OF_SALE
18235,0,0,1,0,0,1,0,0,0,0,0,1,1,0,5913,878,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18239,0,0,1,0,0,1,0,0,0,0,0,1,1,0,5488,48,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18244,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5936,31,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18280,1,0,0,0,0,1,0,0,0,0,0,1,1,0,7813,24,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18285,1,0,0,0,0,1,0,0,0,0,0,1,1,0,8831,160,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,0,1,0,0,0,1,0,0,0,0,0,1,1,0,13215,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
23031,0,1,0,0,0,1,0,0,0,0,0,1,1,0,11612,73,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
23032,0,1,0,0,0,1,0,0,0,0,0,1,1,0,11808,50,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
23033,0,1,0,0,0,1,0,0,0,0,0,1,1,0,12295,23,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


**Do feature selection with SelectKBest.**

In [27]:
# TODO: Select the 15 features that best correlate with the target
# (15 is an arbitrary starting point here)
from sklearn.feature_selection import SelectKBest, f_regression

# SelectKBest has a similar API to what we've seen before.
selector = SelectKBest(score_func=f_regression, k=15)

# IMPORTANT!
# .fit_transform on the train set
# .transform on test set
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


((2507, 15), (644, 15))

**Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)**

In [28]:
import seaborn as sns
x = train['SALE_PRICE']
y = train['BOROUGH']
train.plot.scatter('x', 'y');

KeyError: ignored

In [0]:
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

In [0]:
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=alphas, normalize=True)
ridge.fit(train[['SALE_PRICE']], train['BOROUGH'])
ridge.alpha_

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

**Get mean absolute error for the test set.**

In [0]:
# TODO: How many features should be selected?

# You can try a range of values for k,
# then choose the model with the best score.
# If multiple models "tie" for the best score,
# choose the simplest model.
# You decide what counts as a tie!

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')